In [1]:
import numpy as np
import scipy 
from scipy.integrate import solve_ivp,RK23
import matplotlib.pyplot as plt
import time
from opt_einsum import contract
from scipy.sparse import csr_array,lil_array,csc_array

In [2]:
class MRISolver2D:
    def __init__(self,Nx,Nz,Lx,Lz,ppc=1,sb=3*np.pi,va=1/20,wc=33,mi=10,Nd=3,random_state=None,
                 max_iter=100000,test=False,timer=True,noout=False,
                 pcharge=None,init_pos=None,current=True,weight_order=1,
                 opteinsum=False,sparse=False,nomat=False,vectorweight=True,nomatvec=True):
        """
        Initialize kinetic 2D MRI solver

        Arguments:

        Nx - number of cells in x direction
        Nz - number of cells in z direction
        Lx - length of grid in x direction
        Lz - length of grid in z direction
        sb - shearing parameter normalized to orbital frequency w0/2pi
        va - background magnetic field Alfvén speed normalized to c
        wc - plasma magnetization, ion cyclotron frequency divided by w0
        mi - ion mass over electron mass
        current - Boolean to retain the "displacement current" with  a kinetic calculation (True)
            or assume c curl B - 4pi J = 0
            Default True.
        max_iter - number of iterations after which to stop
        test - Boolean to print information every time step for close debugging.
            Default is False.
        timer - Boolean to keep track of time for simulation components for first two rhs evaluations.
            Default is True.
        noout - Boolean to completely silence the simulation.
            Default is False.
        pcharge - option for initial condition charges. 
            Default (None) randomly selects particle charges
            -1: only initialzes electrons
            1: only initializes ions
        init_pos - species initial position of particles
            Default (None) randomly places particles anywhere in the simulation grid
            "grid": places particles on the grid lattice directly
        weight_order - order of weighting to be used in PIC simulation
            0: particles are only treated as being present in one cell (unstable effects prone to occur)
            1: particles are treated as having a square area equal to one grid cell, contribute to the current or charge
                density of a grid cell based on the fraction of their area in the cell, and feel electric and magnetic fields
                from cells based on the fraction of their area within that cell
        opteinsum - Boolean to use the opt_einsum package to calculate necessary weighting product. 
            Default False.
        sparse - Boolean to use a scipy sparse array for the weighting matrix.
            Default False. 
            The process of weight creation is slower for this form, but it overtakes NumPy array performance as grid points increase.
        nomat - Boolean to only track the indices of the grid particles occupy rather than the weight matrix. 
            Default False. 
            When calcluations are vectorized, this option has potential to be the fastest calculation outlined, but it seems to be unstable for the moment.
        vectorweight - Boolean on whether to use the vectorized form of the weighting calculation.
            Default is False.
        nomatvec - Boolean on whether to use the vectorized form of the particle electromagnetic field or current calculation for the nomat option.
            Default is False.       
        """

        self.random_state = random_state

        self.sb = sb
        self.va = va
        self.wc = wc
        self.mi = mi

        self.Nx = Nx
        self.Nz = Nz
        self.ppc = ppc

        self.Nc = self.Nx*self.Nz
        self.Np = self.Nc * self.ppc

        self.Nd = Nd

        self.Lx = Lx
        self.Lz = Lz
        
        self.xgrid = np.arange(0,Lx,Lx/Nx)
        self.zgrid = np.arange(0,Lz,Lz/Nz)
        self.dr = self.xgrid[1] - self.xgrid[0]
        self.dz = self.zgrid[1] - self.zgrid[0]

        self.max_iter = max_iter
        self.n_iter = 0
        self.test = test
        self.pr = None
        
        self.ZZ,self.XX = np.meshgrid(self.zgrid,self.xgrid)

        """Determine whether to handle current with particles or a nonrelativistic limit""" 
        self.current = current
        
        self.weights = None
        """nearest grid point (NGP) simulation or cloud in cell (CIC/PIC) simulation"""
        self.weight_order = weight_order
        """Options for weight calculation"""
        self.opteinsum = opteinsum
        self.sparse = sparse
        self.nomat = nomat
        self.vectorweight = vectorweight
        self.nomatvec = nomatvec
        
        """Initialize particle charges and masses"""
        self.pcharge = pcharge
        if pcharge == None:
            self.qqs = np.random.choice([1,-1],size=self.Np)
        elif pcharge == -1:
            self.qqs = -np.ones(Nx*Nz*ppc)
        elif pcharge == 1:
            self.qqs = np.ones(Nx*Nz*ppc)
        ion = np.nonzero(self.qqs+1)
        self.mms = np.ones(self.Np)
        self.mms[ion] = self.mi
        
        """Initialize particle positions"""
        self.xxs = np.zeros([self.Np,self.Nd])
        self.init_pos = init_pos
        if init_pos == None:
            self.xxs[:,0] = np.random.uniform(low=0.0,high=Lx,size=self.Np)
            self.xxs[:,2] = np.random.uniform(low=0.0,high=Lz,size=self.Np)
            self.xxs[:,1] = 0.0
        elif init_pos == "grid":
            for i in range(0,ppc):
                self.xxs[i*self.Nx*self.Nz:(i+1)*self.Nx*self.Nz,0] = self.XX.flatten()
                self.xxs[i*self.Nx*self.Nz:(i+1)*self.Nx*self.Nz,1] = 0
                self.xxs[i*self.Nx*self.Nz:(i+1)*self.Nx*self.Nz,2] = self.ZZ.flatten()


        if self.nomat:
            self.xi = np.zeros(self.Np,dtype='int8')
            self.zi = np.zeros(self.Np,dtype='int8')
            self.xj = np.zeros(self.Np,dtype='int8')
            self.zj = np.zeros(self.Np,dtype='int8')
            self.xfrac = np.zeros(self.Np)
            self.zfrac = np.zeros(self.Np)
        self.timer = timer
        self.noout = noout
        
        return(None)
        
    def _advance_positions(self,shaped_xv):
        """
        Use velocities of the particles to move the grid
        """
        vv = shaped_xv[1,:,:]
        """Reshape particle velocities to eventually match 2 x Np x Nd shape"""
        pvv = np.reshape(vv,np.concatenate((np.array([1]),np.shape(vv))))
        return(pvv)
    
    def _particle_grid_weights(self,shaped_xv):
        """Determine weighted position of each particle in the grid
        Arguments:
        shaped_xv - 2 x Np x Nd array with particle positions on first dimension"""

        if self.timer and self.n_iter <= 1:
            tt0 = time.time()

        """Initialize array of weights"""
        if self.sparse:
            """Use a sparse matrix class in this option to store weights
            There are only four nonzero weights for each particle."""
            self.weights = lil_array((self.Np,self.Nx*self.Nz))
        else:
            self.weights = np.zeros([self.Np,self.Nx,self.Nz])
        if self.timer and self.n_iter <= 1:
            tt1 = time.time()
            print("To make LIL ",tt1-tt0)
        
        
        """0th order weighting: only consider current grid cell"""
        if self.weight_order == 0:
            # particle only contributes to where it is
            for i in range(self.Np):
                xi = int(shaped_xv[0,i,0]//self.dr)
                zi = int(shaped_xv[0,i,2]//self.dz)
                self.weights[i,xi,zi] = 1

        """1st order weighting - consider praticle centered in its own grid
        Weights determined by the fraction of this grid 
        in the grid of the electromagnetic fields"""
        
        elif self.weight_order == 1:
            # linear interpolation weights from Birdsall-Langdon            

            if self.vectorweight:
                x = (shaped_xv[0,:,0]/self.dr)
                z = (shaped_xv[0,:,0]/self.dz)
                xi = x.astype(int)
                zi = z.astype(int)
                """Find x and z indices that the particle overlaps with"""
                try:
                    xj = xi + (x-xi-0.5)/np.abs(x-xi-0.5)
                except:
                    xj = xi -1
                try:
                    zj = zi + (z-zi-0.5)/np.abs(z-zi-0.5)
                except:
                    zj = zi - 1

                xj = np.mod(xj,self.Nx)
                zj = np.mod(zj,self.Nz)
                xj = xj.astype(int)
                zj = zj.astype(int)
                
                """Vectorized weights"""
                xfrac = 1 + np.abs(x-xi-0.5)
                zfrac = 1 - np.abs(x-xi-0.5)

                """Save weights into simulation class"""
                if self.nomat:
                    self.xi = xi
                    self.zi = zi
                    self.xj = xj
                    self.zj = zj
                    self.xfrac = xfrac
                    self.zfrac = zfrac
                elif not self.sparse:
                    self.weights[:,xi,zi] = xfrac*zfrac
                    self.weights[:,xj,zi] = (1-xfrac)*zfrac
                    self.weights[:,xi,zj] = xfrac*(1-zfrac)
                    self.weights[:,xj,zj] = (1-xfrac)*(1-zfrac)
                else:
                    ind1 = self.Nz*xi + zi
                    ind2 = self.Nz*xj + zi
                    ind3 = self.Nz*xi + zj
                    ind4 = self.Nz*xj + zj
                    self.weights[:,ind1] = xfrac*zfrac
                    self.weights[:,ind2] = (1-xfrac)*zfrac
                    self.weights[:,ind3] = xfrac*(1-zfrac)
                    self.weights[:,ind4] = (1-xfrac)*(1-zfrac)
            
            else:          
                for i in range(self.Np):
                    """Find grid cell particle is in"""
                    xi = np.mod(int(shaped_xv[0,i,0]//self.dr),self.Nx)
                    zi = np.mod(int(shaped_xv[0,i,2]//self.dz),self.Nz)
                
                    # print((shaped_xv[0,i,0]-self.xgrid[xi])/self.dr)
                    # print((shaped_xv[0,i,2]-self.zgrid[zi])/self.dz)
                    
                    """Determnine weights for particle and find grid points 
                    particle overlaps with"""
                    if (shaped_xv[0,i,0]-self.xgrid[xi])/self.dr < 0.5:
                        xfrac = 0.5 + (shaped_xv[0,i,0]-self.xgrid[xi])/self.dr
                        xj = np.mod(xi-1,self.Nx)
                    else:
                        xfrac = 1.5 - (shaped_xv[0,i,0]-self.xgrid[xi])/self.dr
                        xj = np.mod(xi+1,self.Nx)
                    if (shaped_xv[0,i,2]-self.zgrid[zi])/self.dz < 0.5:
                        zfrac = 0.5 + (shaped_xv[0,i,2]-self.zgrid[zi])/self.dz
                        zj = np.mod(zi-1,self.Nz)
                    else:
                        zfrac = 1.5 - (shaped_xv[0,i,2]-self.zgrid[zi])/self.dz
                        zj = np.mod(zi+1,self.Nz)
                        
                    """Save weights into simulation class"""
                    if self.nomat:
                        self.xi[i] = int(xi)
                        self.zi[i] = int(zi)
                        self.xj[i] = int(xj)
                        self.zj[i] = int(zj)
                        self.xfrac[i] = xfrac
                        self.zfrac[i] = zfrac
                    elif not self.sparse:
                        self.weights[i,xi,zi] = xfrac*zfrac
                        self.weights[i,xj,zi] = (1-xfrac)*zfrac
                        self.weights[i,xi,zj] = xfrac*(1-zfrac)
                        self.weights[i,xj,zj] = (1-xfrac)*(1-zfrac)
                    else:
                        ind1 = self.Nz*xi+zi
                        ind2 = self.Nz*xj+zi
                        ind3 = self.Nz*xi+zj
                        ind4 = self.Nz*xj+zj
                        self.weights[i,ind1] = xfrac*zfrac
                        self.weights[i,ind2] = (1-xfrac)*zfrac
                        self.weights[i,ind3] = xfrac*(1-zfrac)
                        self.weights[i,ind4] = (1-xfrac)*(1-zfrac)
                        # if self.test:
                        # print("xfrac ",xfrac," ",1-xfrac)
                        # print("zfrac ",zfrac," ",1-zfrac)
        
        if self.timer and self.n_iter <= 1:
            tt2 = time.time()
            print("To Set Weights ",tt2-tt1)
                
        else:
            return(NotImplementedError)
        # if self.pr:
        #     print("Weights Max ",np.amax(self.weights))
            
        return(None)
    
     # Construct field on particles from np.einsum
        
    def _fields_on_particle(self,shaped_eb):

        """Find electric and magnetic fields at each particle position"""
        if self.timer and self.n_iter <= 1:
            tt0 = time.time()
        
        if self.nomat:
            EB = np.zeros([2,self.Np,self.Nd])
            
            """Only sum over four grid points the finite-sized particle could be in"""
            if not self.nomatvec:
                for i in range(self.Np):
                    EB[:,i,:] = self.xfrac[i]*self.zfrac[i]*shaped_eb[:,self.xi[i],self.zi[i],:]
                    EB[:,i,:] += (1-self.xfrac[i])*self.zfrac[i]*shaped_eb[:,self.xj[i],self.zi[i],:]
                    EB[:,i,:] += self.xfrac[i]*(1-self.zfrac[i])*shaped_eb[:,self.xi[i],self.zj[i],:]
                    EB[:,i,:] += (1-self.xfrac[i])*(1-self.zfrac[i])*shaped_eb[:,self.xj[i],self.zj[i],:]

            """Vectorized calculation - may not be stable yet"""
            else:
                EB = self.xfrac[None,:,None] * self.zfrac[None,:,None] * shaped_eb[:,self.xi,self.zi,:]
                EB += (1-self.xfrac[None,:,None]) * self.zfrac[None,:,None] * shaped_eb[:,self.xj,self.zi,:]
                EB += self.xfrac[None,:,None] * (1-self.zfrac[None,:,None]) * shaped_eb[:,self.xi,self.zj,:]
                EB += (1-self.xfrac[None,:,None])*(1-self.zfrac[None,:,None]) * shaped_eb[:,self.xj,self.zj,:]

            E = EB[0,:,:]
            B = EB[1,:,:]

        elif self.sparse:
            """Convert the electromagnetic field grid components to enumeration of grid points"""
            eb_sparse = np.reshape(shaped_eb[:,:,:,:],[2,self.Nx*self.Nz,3])
            """Convert the sparse matrix to a csr matrix for faster computation
            Do the matrix multiplication to find weights"""
            E = self.weights.tocsr() @ eb_sparse[0,:,:]
            B = self.weights.tocsr() @ eb_sparse[1,:,:]
        elif self.opteinsum:
            """Use opt_einsum to do the best below einsum"""
            E = contract("ijk,jkl",self.weights,shaped_eb[0,:,:,:])
            B = contract("ijk,jkl",self.weights,shaped_eb[1,:,:,:])
        else:
            """Use einsum to sum over needed indices to find the electromagnetic fields
            relevant for a particle"""
            E = np.einsum("ijk,jkl",self.weights,shaped_eb[0,:,:,:],optimize=True)
            B = np.einsum("ijk,jkl",self.weights,shaped_eb[1,:,:,:],optimize=True)
        '''Explore more efficient options here too - likely only a few particles at one grid cell are relevant'''

        if self.timer and self.n_iter <= 1:
            tt1 = time.time()
            print("Time to get fields ",tt1-tt0)
        return E,B
        
    def _current_density(self,shaped_xv):

        if self.timer and self.n_iter <= 1:
            tt0 = time.time()

        if self.nomat:
            """"""
            J = np.zeros([self.Nx,self.Nz,self.Nd])
            if not self.nomatvec:
                for i in range(self.Np):
                    J[self.xi[i],self.zi[i],:] += self.qqs[i]*shaped_xv[1,i,:]*self.xfrac[i]*self.zfrac[i]
                    J[self.xj[i],self.zi[i],:] += self.qqs[i]*shaped_xv[1,i,:]*(1-self.xfrac[i])*self.zfrac[i]
                    J[self.xi[i],self.zj[i],:] += self.qqs[i]*shaped_xv[1,i,:]*self.xfrac[i]*(1-self.zfrac[i])
                    J[self.xj[i],self.zj[i],:] += self.qqs[i]*shaped_xv[1,i,:]*(1-self.xfrac[i])*(1-self.zfrac[i])
            else:
                J[self.xi,self.zi,:] += self.qqs[:,None] *shaped_xv[1,:,:] * self.xfrac[:,None] * self.zfrac[:,None]
                J[self.xj,self.zi,:] += self.qqs[:,None] * shaped_xv[1,:,:] *  (1-self.xfrac[:,None]) * self.zfrac[:,None]
                J[self.xi,self.zj,:] += self.qqs[:,None] * shaped_xv[1,:,:]  * self.xfrac[:,None] * (1-self.zfrac[:,None])
                J[self.xj,self.zj,:] += self.qqs[:,None] * shaped_xv[1,:,:] * (1-self.xfrac[:,None])*(1-self.zfrac[:,None])
                
        elif self.sparse:
            """Use the converted sparse weight matrix to contract over particles"""
            J_sparse = (self.weights.tocsr()).transpose() @ (self.qqs[:,None] * shaped_xv[1,:,:])
            # np.einsum("i,ij,il",self.qqs,self.weights,shaped_xv[1,:,:],optimize=True)
            """Reshape the resulting (Nx Nz) x Nd matrix to a current grid Nx x Nz x Nd"""
            J = np.reshape(J_sparse,[self.Nx,self.Nz,self.Nd])
        elif self.opteinsum:
            """Use opt_einsum to detect the fastest contraction"""
            J = contract("i,ijk,il",self.qqs,self.weights,shaped_xv[1,:,:])
        else:
            """Use einsum to sum over particle charges, velocities, and weights of where particles are"""
            J = np.einsum("i,ijk,il",self.qqs,self.weights,shaped_xv[1,:,:],optimize=True)
            
        """Normalize J"""
        J *= 1/self.va * self.wc * 1/2

        if self.timer and self.n_iter <= 1:
            tt1 = time.time()
            print("Time for Current ",tt1-tt0)
            
        if self.pr:
            print("Current Max ",np.amax(J))

        return(J)
        
    
    def _charge_density(self):
        
        """This code could be used to find the charge density on the grid
        in an analogous way to the current density.
        This isn't being used the current version of the code but could be used to
        include the charge density in a current calculation"""
        if self.opteinsum:
            cd = contract("i,ijk",self.qqs,self.weights)
        else:
            cd = np.einsum("i,ijk",self.qqs,self.weights,optimize=True)
        
        return(cd)
    
    
    def _lorentz_force(self,shaped_xv,shaped_eb,i):
        """
        Computes components of the Lorentz force as
        2pi * qalpha/|e| * mi/malpha * 1/self.va * self.wc * (E + self.va * u x B)
        Arguments:
            shaped_y - evolved quantities
            i - component of Lorentz force to evaluate
        """
        lorentz = np.zeros([1,self.Np,1])
        
        """Essentially find the Levi-Civita tensor"""
        if i == 0:
            j = 1
            k = 2
        elif i == 1:
            j = 2
            k = 0
        else:
            j = 0
            k = 1
        
        E,B = self._fields_on_particle(shaped_eb)
        """Calculate v x B"""
        lorentz = shaped_xv[1,:,j]*B[:,k]
        lorentz -= shaped_xv[1,:,k]*B[:,j]
        """Add E"""
        lorentz += E[:,i]/self.va

        """Normalize"""
        lorentz *= 2*np.pi*self.qqs[:]/self.mms[:]
        lorentz *=  self.wc*self.mi

        if self.pr:
            print("Max lorentz",np.amax(lorentz))

        return(lorentz)

    def _advance_vs(self,shaped_xv,shaped_eb):
        """
        Advances particle velocities according to
        du/dt = 4pi uy \hat{x} - pi ux \hat{y} + 2pi * qalpha/|e| * mi/malpha * 1/self.va * self.wc * (E + self.va * u x B)
        """
        momeqn = np.zeros([1,self.Np,self.Nd])
        momeqn[:,:,0] = self._lorentz_force(shaped_xv,shaped_eb,0)+4*np.pi * shaped_xv[1,:,1]
        momeqn[:,:,1] = self._lorentz_force(shaped_xv,shaped_eb,1)-np.pi*shaped_xv[1,:,0]
        momeqn[:,:,2] = self._lorentz_force(shaped_xv,shaped_eb,2)

        return(momeqn)

    def _curl(self,field):
        """
        Finds curl components in polar coordinates of field
        radial component = - dfield_phi/dz
        phi component = dfield_r/dz-dfield_z/dr
        z component = dfield_phi/dr 
        (assumed no angular dependence)
        Arguments:
            field - field whose curl to compute
            position - grid position corresponding to field location
        """

        """Fully periodic boundary conditions enforced by using 9 copies of our input field array"""
        curl_field = np.zeros([1,self.Nx,self.Nz,self.Nd])
        pfield = np.reshape(field,np.concatenate((np.array([1]),np.shape(field))))
        xcat = np.concatenate((pfield,pfield,pfield),axis=1)
        zcat = np.concatenate((xcat,xcat,xcat),axis=2)

        """Centered differencing for spatial derivatives"""
        curl_field[0,:,:,0] = - (zcat[0,self.Nx:2*self.Nx,self.Nz+1:2*self.Nz+1,1]-zcat[0,self.Nx:2*self.Nx,self.Nz-1:2*self.Nz-1,1])/(2*self.dz)
        curl_field[0,:,:,1] = (zcat[0,self.Nx:2*self.Nx,self.Nz+1:2*self.Nz+1,0]-zcat[0,self.Nx:2*self.Nx,self.Nz-1:2*self.Nz-1,0])/(2*self.dz)
        curl_field[0,:,:,1] -= (zcat[0,self.Nx+1:2*self.Nx+1,self.Nz:2*self.Nz,2]-zcat[0,self.Nx-1:2*self.Nx-1,self.Nz:2*self.Nz,2])/(2*self.dr)
        curl_field[0,:,:,2] = (zcat[0,self.Nx+1:2*self.Nx+1,self.Nz:2*self.Nz,1]-zcat[0,self.Nx-1:2*self.Nx-1,self.Nz:2*self.Nz,1])/(2*self.dr)

        if self.pr:
            print("Curl z Max",np.amax(curl_field[:,:,:,2]))
            print("Curl x Max",np.amax(curl_field[:,:,:,0]))
            print("Curl y Max",np.amax(curl_field[:,:,:,1]))
            
        return(curl_field)
        
    def _advance_fields(self,shaped_xv,shaped_eb):
        """
        Implements the shearing frame Maxwell equations
        dB/dt = -1/self.va curl E - self.sb Bx \hat{y}
        dE/dt = 1/self.va curl B + 4pi J - self.sb Ex \hat{y}
        J found from the particle velocities
        """

        """Get curl of E and B"""
        E_curl = self._curl(shaped_eb[0,:,:,:])
        B_curl = self._curl(shaped_eb[1,:,:,:])

        deriv = np.zeros([2,self.Nx,self.Nz,self.Nd])

        """Get current density"""
        J = self._current_density(shaped_xv)
        deriv[0,:,:,:] = 1/self.va * B_curl - 4 * np.pi * J
        
        if self.current==False: # allow to ignore displacement current
            deriv[0,:,:,:] = 0
        
        deriv[0,:,:,1] -= self.sb * shaped_eb[0,:,:,0]
        
        deriv[1,:,:,:] = - 1/self.va * E_curl
        deriv[1,:,:,1] -= self.sb * shaped_eb[1,:,:,0]
        if self.pr:
            print("Max derivex",np.amax(deriv[0,:,:,0]))
            print("Max derivey",np.amax(deriv[0,:,:,1]))
            print("Max derivez",np.amax(deriv[0,:,:,2]))
            print("Max derivbx",np.amax(deriv[1,:,:,0]))
            print("Max derivby",np.amax(deriv[1,:,:,1]))
            print("Max derivbz",np.amax(deriv[1,:,:,2]))
        return(deriv)

    def _rhs(self,t,y):
        """
        Stacks grid, velocity, and fields
        Finds right hand sides of EOMs
        """

        if self.test:
            self.pr = True
        elif self.n_iter % 100 < 1:
            self.pr = True
        else:
            self.pr = False
        if self.noout == True:
            self.timer = False
            self.pr = False
        if self.pr:
            print("time = ",t)
        if self.timer and self.n_iter <= 1:
            t0 = time.time()
            
        shaped_xv = np.reshape(y[:2*self.Np*self.Nd],[2,self.Np,self.Nd])
        shaped_eb = np.reshape(y[2*self.Np*self.Nd:],[2,self.Nx,self.Nz,self.Nd])

        """Make sure particles have not overshot the grid
        Reset x, z to within 0, Lx ; 0, Lz; y to within 0, 2Pi"""

        shaped_xv[0,:,0] = np.mod(shaped_xv[0,:,0],self.Lx)
        shaped_xv[0,:,1] = np.mod(shaped_xv[0,:,1],2*np.pi)
        shaped_xv[0,:,2] = np.mod(shaped_xv[0,:,2],self.Lz)

        """Calculate right hand sides of equations"""
        
        self._particle_grid_weights(shaped_xv)
        if self.timer and self.n_iter <= 1:
            t1 = time.time()      
            print("Time to Weight ",t1-t0)
        xderiv = self._advance_positions(shaped_xv)
        if self.timer and self.n_iter <= 1:
            t2 = time.time()
            print("Time for XDeriv ",t2-t1)
        vderiv = self._advance_vs(shaped_xv,shaped_eb)
        if self.timer and self.n_iter <= 1:
            t3 = time.time()
            print("Time for VDeriv ",t3-t2)
        
        EBderiv = self._advance_fields(shaped_xv,shaped_eb)
        if self.timer and self.n_iter <= 1:
            t4 = time.time()
            print("Time for Concat and EBDeriv ",t4-t3)
        
        """Reshape arrays to match solve_ivp rhs output"""
        xv_deriv = np.concatenate((xderiv,vderiv),axis=0)
        flat_total = np.concatenate((xv_deriv.flatten(),EBderiv.flatten()))

        self.n_iter += 1
        if (self.n_iter > self.max_iter):
            exit()
        
        return(flat_total)
        
    def solve(self,vs,es,bs,t0=0,tf=20,NT=11,method="DOP853"):
        """
        Formats listed positions and velocities into a 1D vector which solve_ivp integrates
        Arguments:
        xs - particle initial positions to evolve ( (Nx x Nz x ppc) x 3)
        vs - particle initial velocities ((Nx Nz ppc) x 3)
        es - initial electric field at grid positions (Nx x Nz x 3)
        bs - initial magnetic field at grid positions (Nx x Nz x 3)
        """

        
        vvs = np.reshape(vs,[self.Np,self.Nd])
        ees = np.reshape(es,[self.Nx,self.Nz,self.Nd])
        bbs = np.reshape(bs,[self.Nx,self.Nz,self.Nd])
        
        shaped_xv0 = np.stack((self.xxs,vvs),axis=0)
        shaped_eb0 = np.stack((ees,bbs),axis=0)
        y0 = np.concatenate((shaped_xv0.flatten(),shaped_eb0.flatten()))
        
        t_span = np.linspace(t0,tf,NT)
        sol = solve_ivp(self._rhs,[t0,tf],y0,t_eval=t_span)
        print("Finish Status ",sol.status)
        reshaped_xv = np.reshape(sol.y[:2*self.Np*self.Nd,:],[2,self.Np,self.Nd,NT])
        reshaped_eb = np.reshape(sol.y[2*self.Np*self.Nd:,:],[2,self.Nx,self.Nz,self.Nd,NT])
        
        return(sol.t,reshaped_xv,reshaped_eb)
        

In [3]:
Nx = 5
Nz = 100
Lx = Nx*0.1
Lz = Nz*0.1

solver = MRISolver2D(Nx,Nz,Lx,Lz,ppc=1,sb=3*np.pi,va=1/20,wc=33,mi=10,
                     Nd=3,max_iter=100000,pcharge=None,random_state=None,noout=False,
                     test=False,current=True,weight_order=1,init_pos="grid",opteinsum=False,sparse=False,timer=True,
                     nomat=False,vectorweight=False,nomatvec=True)


"""Create initial velocities
Riquelme seeded velocity 1/20 sin(2 pi z/Lz) \hat{x}"""
vx = 1/20 * np.sin(2 * np.pi * solver.xxs[:,2]/Lz)
vy = np.zeros(solver.Np)
vz = np.zeros(solver.Np)
vvs = np.stack((vx,vy,vz),axis=-1)

# add noise
#vvs += solver.va/100 * np.random.normal(np.shape(vvs)[1:5])

# Create initial electric field
# start no electric field for unknown reasons
ees = np.zeros([solver.Nx,solver.Nz,solver.Nd])

"""option to add noise to no initial electric field"""
#ees += np.random.normal(np.shape(ees)[1:4])*solver.va/100

# later - field due to particles

"""# Create initial magnetic field
background magnetic field"""
bbs = np.zeros([solver.Nx,solver.Nz,solver.Nd])
bbs[:,:,2] = 0.4 * (solver.mi*solver.wc)**(-1) #  0.5 - 2.0 prefactors are unstable for nonrelativistic current
bbs[:,:,2] = 1 # Initial condition for VAz0 = 1

"""option to add noise to initial magnetic field"""
#bbs += np.random.normal(np.shape(bbs)[1:4])*solver.va/100



In [4]:
solution = solver.solve(vvs,ees,bbs,t0=0,tf=4,method="DOP853")
print(solver.n_iter)
t = solution[0]
xv = solution[1]
EB = solution[2]
np.savez("mrielectron5x100_114",t=t,xv=xv,EB=EB,Nx=solver.Nx,Nz=solver.Nz,Lx=solver.Lx,Lz=solver.Lz,ppc=solver.ppc)

time =  0.0
To make LIL  8.225440979003906e-05
To Set Weights  0.012441873550415039
Time to Weight  0.012759923934936523
Time for XDeriv  7.700920104980469e-05
Time to get fields  0.012431859970092773
Max lorentz -0.0
Time to get fields  0.0010559558868408203
Max lorentz 103.67255756846318
Time to get fields  0.0011489391326904297
Max lorentz -0.0
Time for VDeriv  0.016550064086914062
Curl z Max 0.0
Curl x Max -0.0
Curl y Max 0.0
Curl z Max 0.0
Curl x Max -0.0
Curl y Max 0.0
Time for Current  0.005332231521606445
Current Max  16.4837205095333
Max derivex 207.14054102630885
Max derivey 0.0
Max derivez 0.0
Max derivbx 0.0
Max derivby -0.0
Max derivbz -0.0
Time for Concat and EBDeriv  0.009033918380737305
To make LIL  0.00045108795166015625
To Set Weights  0.013781070709228516
Time to Weight  0.014739990234375
Time for XDeriv  0.00011110305786132812
Time to get fields  0.0010840892791748047
Time to get fields  0.0007691383361816406
Time to get fields  0.0005979537963867188
Time for VDeriv